### Código procedente de Kaggle.
***Autor:***
* **LIU SHI CAI · 5Y AGO**

In [11]:
# %pip install keras
# %pip install tensorflow

In [12]:
import numpy as np
import pandas as pd
# import seaborn as sns
from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt
# from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

In [14]:
# leemos los archivos CSV utilizando la biblioteca pandas.
# Los datos se almacenan en dos DataFrames: train_df y test_df.
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [29]:
# Separamos los datos de entrenamiento en dos matrices NumPy diferentes: labels y train.
labels = train_df[:,0]

# Se dividen por 255 para realizar una normalización de los píxeles, asumiendo que los valores originales están en el rango de 0 a 255.
train = train_df[,1:]/255

SyntaxError: invalid syntax (3832516617.py, line 5)

In [30]:
# La función to_categorical de Keras se utiliza para convertir las etiquetas numéricas en codificación one-hot.
dummy_y = keras.utils.to_categorical(labels)

# La función train_test_split de la biblioteca scikit-learn se utiliza para dividir los datos en conjuntos de entrenamiento y prueba.
# Se utiliza un tamaño de prueba del 10% (test_size=0.1).
# Se establece una semilla aleatoria (random_state=166) para que la división sea reproducible.
# La opción stratify=labels garantiza que las proporciones de las clases se mantengan en los conjuntos de entrenamiento y prueba.
x_train, x_test, y_train, y_test = train_test_split(train, dummy_y, test_size=0.1, random_state=166,stratify=labels)

MemoryError: Unable to allocate 31.4 GiB for an array with shape (32970000, 256) and data type float32

In [ ]:
'''
Las matrices x_train y x_test se remodelan utilizando el método reshape para adaptarse a un formato específico antes de ser utilizadas 
en un modelo de clasificación de imágenes.

La forma original de los datos en MNIST es de imágenes en escala de grises de 28x28 píxeles. Sin embargo, para entrenar un modelo de 
aprendizaje profundo, generalmente se requiere que los datos estén en un formato específico, que es
[número_de_ejemplos, ancho, alto, canales].

En este caso, se está agregando una dimensión adicional a los datos utilizando reshape. El primer argumento x_train.shape[0] indica el número
de ejemplos de entrenamiento, y luego se especifica la forma deseada de las imágenes, que es (28, 28, 1). La dimensión adicional de 1 se refiere
al canal de color y representa la escala de grises.
'''
# Remodelar los datos de entrenamiento y prueba.
x_train = x_train.reshape(x_train.shape[0],28,28, 1)

# El ancho y alto son 28x28 píxeles.
# Y los canales son 1, que representa la escala de grises.
x_test = x_test.reshape(x_test.shape[0],28,28, 1)

'''
Dado que el módulo de lime que estamos utilizando solo funciona con imágenes en 3D, es decir, una imagen que tiene 3 canales generalmente
RGB, replicamos el plano en escala de grises aquí. Este segmento de código convierte una imagen en escala de grises a RGB simplemente
replicando el plano disponible.
'''
# Creamos la función que adapta los datos.
import numpy as np
def to_rgb(x):
    x_rgb = np.zeros((x.shape[0], 28, 28, 3))
    for i in range(3):
        x_rgb[..., i] = x[..., 0]
    return x_rgb

# Convertimos los datos al formato que posteriormente necesitaremos.
x_train = to_rgb(x_train)
x_test = to_rgb(x_test)

In [ ]:
# Definimos y entrenamos un modelo de red neuronal convolucional utilizando Keras.


# Creamos una instancia del modelo secuencial de Keras, que permite agregar capas de manera secuencial.
model = Sequential()

# Se define una lista de callbacks, en este caso se utiliza el ModelCheckpoint para guardar el mejor modelo durante el entrenamiento. 
# El modelo se guarda en el archivo "convolutional_minist.h5" basado en la métrica de precisión en la validación (val_acc).
callbacks = [keras.callbacks.ModelCheckpoint('cnn_minist.h5', monitor='val_accuracy', verbose=1, save_best_only=True,
                            mode='auto')]

'''Se agregan capas convolucionales utilizando la función add() del modelo secuencial. Se utilizan cuatro capas 
  convolucionales con diferentes tamaños de kernel y funciones de activación 'relu'.
'''

# La primera capa tiene un tamaño de kernel de (3,3) y recibe la forma de entrada (28,28,1) para imágenes en escala de grises.
model.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu',padding='same',
                 input_shape=(28,28,3))) # MODIFICAMOS LA ENTRADA RESPECTO DEL ORIGINAL (28, 28, **3**).

# Las siguientes capas convolucionales tienen el mismo tamaño de kernel (3,3) y 'padding' 'same' para mantener el tamaño de la imagen.
model.add(Conv2D(64, (3, 3),padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3),padding='same', activation='relu'))

# La cuarta capa convolucional tiene un tamaño de kernel de (28,28) y se utiliza una activación 'relu'.
model.add(Conv2D(128, (28, 28),activation='relu'))

# Se agrega una capa de aplanamiento (flatten) para convertir los mapas de características en un vector unidimensional.
model.add(Flatten())

# Se agrega una capa densa con 256 neuronas y activación 'relu'.
model.add(Dense(256, activation='relu'))

# Se agrega una capa de dropout con una tasa de dropout del 50% para regularizar el modelo y evitar el sobreajuste.
model.add(Dropout(0.5))

# Se agrega una capa densa final con 10 neuronas (correspondientes a las 10 clases de salida). 
# Una función de activación 'softmax' para la clasificación multiclase.
model.add(Dense(10, activation='softmax'))



# Función utilizada en Keras para mostrar un resumen del modelo de la red neuronal. 
# Nos da una descripción de la arquitectura de la red, la forma de entrada y salida de las capa y número de parámetros entrenables del modelo.
model.summary()

# Se compila el modelo especificando la función de pérdida ('categorical_crossentropy').
# Se define además, el optimizador ('sgd' con una tasa de aprendizaje de 0.01 y momentum de 0.9) y las métricas a utilizar ('accuracy').

sgd = SGD(learning_rate=0.01, momentum=0.9) # "lr" en desuso cambiarlo por "learning_rate".

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Se ajusta el modelo utilizando los datos de entrenamiento.
# Se especifica el tamaño del lote, el número de épocas, la verbosidad y los datos de validación (x_test y y_test). 
# También se pasan los callbacks definidos anteriormente.
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test),callbacks=callbacks)

In [ ]:
# Función que se utiliza para cargar los pesos pre-entrenados de un modelo guardado en un archivo HDF5.
model.load_weights('cnn_minist.h5')

In [ ]:
# Le damos el formato adecuado a los datos de prueba (test) antes de realizar predicciones con el modelo.
# Al realizar esta transformación en los datos de prueba, se asegura que tengan el mismo formato que los datos de entrenamiento.
test = test_data.reshape(test_data.shape[0],28,28, 1)/255

test = to_rgb(test)

In [ ]:
# Obtenemos las predicciones del modelo sobre los datos de prueba.
predict = model.predict(test)

In [ ]:
# Obtenemos las etiquetas predichas a partir de las probabilidades de salida del modelo.
results = np.argmax(predict,axis = 1)

In [ ]:
# Creamos un DataFrame, "cnn_submission" que contiene dos columnas: "ImageId" y "Label". 
# La columna "ImageId" contiene los valores del rango del 1 al 28000 (indicando el número de imagen correspondiente), 
# La columna "Label" contiene las etiquetas predichas para cada imagen.
submission = pd.DataFrame({"ImageId":range(1,28001),"Label":results})
submission.to_csv("cnn_submission.csv",index=False)

In [ ]:
# Publicamos en pantalla el DataFrame "submission".
submission

In [ ]:
# Comprobamos que el modelo se ha guardado adecuadamente.
cnn_minist = load_model('cnn_minist.h5')

In [ ]:
cnn_minist.summary()